In [ ]:
print("📦 Installing required packages...") #installing imp packages  MY TRAINED MODEL IS SAVED in-  /content/drive/MyDrive/FinIQ_Model
print("⏱️  This will take 2-3 minutes\n")

!pip install -q transformers datasets torch scikit-learn

print("\n✅ All packages installed successfully!")
print("🎉 Ready to proceed!")

📦 Installing required packages...
⏱️  This will take 2-3 minutes


✅ All packages installed successfully!
🎉 Ready to proceed!


In [ ]:
import pandas as pd #preparing the dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
import os

print("📂 Loading your CSV file...")
df = pd.read_csv('all-data.csv', encoding='latin-1', names=['sentiment', 'sentence'])

print(f"✅ Loaded {len(df)} examples")
print(f"\nFirst few rows:")
print(df.head())

# Clean the data
print("\n🧹 Cleaning data...")
sentiment_map = {
    'positive': 2,
    'neutral': 1,
    'negative': 0
}

df['label'] = df['sentiment'].map(sentiment_map)
df = df[['sentence', 'label']].dropna()
df = df.drop_duplicates(subset=['sentence'])

print(f"✅ Cleaned data: {len(df)} examples")
print("\nLabel distribution:")
print(df['label'].value_counts().sort_index())

# Split into train/test
print("\n✂️ Splitting data...")
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['sentence', 'label']], preserve_index=False)
test_dataset = Dataset.from_pandas(test_df[['sentence', 'label']], preserve_index=False)

print(f"✅ Training set: {len(train_dataset)} examples")
print(f"✅ Testing set: {len(test_dataset)} examples")

# Save datasets
os.makedirs('./data', exist_ok=True)
train_dataset.save_to_disk('./data/train')
test_dataset.save_to_disk('./data/test')

print("\n💾 Datasets saved to ./data/")
print("🎉 Dataset preparation complete!")

📂 Loading your CSV file...
✅ Loaded 4846 examples

First few rows:
  sentiment                                           sentence
0   neutral  According to Gran , the company has no plans t...
1   neutral  Technopolis plans to develop in stages an area...
2  negative  The international electronic industry company ...
3  positive  With the new production plant the company woul...
4  positive  According to the company 's updated strategy f...

🧹 Cleaning data...
✅ Cleaned data: 4838 examples

Label distribution:
label
0     604
1    2872
2    1362
Name: count, dtype: int64

✂️ Splitting data...
✅ Training set: 3870 examples
✅ Testing set: 968 examples


Saving the dataset (0/1 shards):   0%|          | 0/3870 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/968 [00:00<?, ? examples/s]


💾 Datasets saved to ./data/
🎉 Dataset preparation complete!


In [ ]:
from google.colab import drive

print("📁 Mounting Google Drive...")
print("⚠️  A link will appear - click it and authorize\n")

drive.mount('/content/drive')

print("\n✅ Google Drive mounted successfully!")
print("📂 Your Drive is accessible at: /content/drive/MyDrive")

# Create save directory
import os
save_path = '/content/drive/MyDrive/FinIQ_Model'
os.makedirs(save_path, exist_ok=True)

print(f"✅ Model save directory created: {save_path}")
print("⭐ Your model will be permanently saved here!")

📁 Mounting Google Drive...
⚠️  A link will appear - click it and authorize

Mounted at /content/drive

✅ Google Drive mounted successfully!
📂 Your Drive is accessible at: /content/drive/MyDrive
✅ Model save directory created: /content/drive/MyDrive/FinIQ_Model
⭐ Your model will be permanently saved here!


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import gc
import os

# Save path in Google Drive
save_path = '/content/drive/MyDrive/FinIQ_Model'

print("🚀 STARTING MODEL TRAINING")
print("=" * 70)
print("⏱️  Expected time: 15-20 minutes")
print("☕ Grab a coffee and relax!")
print("=" * 70)

# Clear memory
torch.cuda.empty_cache()
gc.collect()

# Load datasets
print("\n📂 Loading datasets...")
train_dataset = load_from_disk('./data/train')
test_dataset = load_from_disk('./data/test')

# Use 2000 examples for balanced speed + accuracy
print("⚡ Using 2000 training examples (balanced approach)...")
train_dataset = train_dataset.select(range(min(2000, len(train_dataset))))
test_dataset = test_dataset.select(range(min(400, len(test_dataset))))

print(f"✅ Training: {len(train_dataset)} examples")
print(f"✅ Testing: {len(test_dataset)} examples")

# Load tokenizer
print("\n🔤 Loading FinBERT tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
print("✅ Tokenizer loaded!")

# Tokenize data
print("\n⚙️ Tokenizing data...")
def tokenize_function(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)
print("✅ Tokenization complete!")

# Load model
print("\n🤖 Loading FinBERT model...")
from transformers import BertConfig

config = BertConfig.from_pretrained("ProsusAI/finbert")
config.num_labels = 3
config.hidden_dropout_prob = 0.1
config.attention_probs_dropout_prob = 0.1

model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert",
    config=config,
    ignore_mismatched_sizes=True
)
print("✅ Model loaded!")

# Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='weighted', zero_division=0
    )

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training configuration
print("\n⚙️ Setting up training configuration...")
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=25,
    save_total_limit=1,
    report_to="none",
    fp16=True,
)

# Create Trainer
print("\n👨‍🏫 Creating trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)
print("✅ Trainer ready!")

# START TRAINING
print("\n" + "=" * 70)
print("🚀 TRAINING STARTED!")
print("=" * 70)

try:
    trainer.train()

    print("\n" + "=" * 70)
    print("🎉 TRAINING COMPLETE!")
    print("=" * 70)

    # Save to Google Drive
    print(f"\n💾 Saving model to Google Drive...")
    print(f"📍 Location: {save_path}")

    trainer.save_model(save_path)
    tokenizer.save_pretrained(save_path)

    # Verify save
    if os.path.exists(save_path):
        files = os.listdir(save_path)
        print(f"\n✅ SUCCESS! Model saved to Google Drive!")
        print(f"📁 Total files saved: {len(files)}")
        print(f"⭐ Your model is PERMANENTLY saved!")
        print(f"⭐ Location: {save_path}")

    # Evaluate
    print("\n📊 FINAL RESULTS:")
    print("=" * 70)
    eval_results = trainer.evaluate()

    for key, value in eval_results.items():
        print(f"{key}: {value:.4f}")

    print("\n" + "=" * 70)
    print("🎉 ALL DONE! YOUR MODEL IS READY!")
    print("=" * 70)
    print(f"📂 Model saved at: {save_path}")
    print("🚀 Next step: Test your model!")
    print("=" * 70)

except Exception as e:
    print(f"\n❌ Error during training: {e}")
    import traceback
    traceback.print_exc()

🚀 STARTING MODEL TRAINING
⏱️  Expected time: 15-20 minutes
☕ Grab a coffee and relax!

📂 Loading datasets...
⚡ Using 2000 training examples (balanced approach)...
✅ Training: 2000 examples
✅ Testing: 400 examples

🔤 Loading FinBERT tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ Model loaded!

⚙️ Setting up training configuration...

👨‍🏫 Creating trainer...
✅ Trainer ready!

🚀 TRAINING STARTED!


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.437000,0.457540,0.850000,0.849350,0.849023,0.850000
2,0.246700,0.401093,0.862500,0.861842,0.861824,0.862500



🎉 TRAINING COMPLETE!

💾 Saving model to Google Drive...
📍 Location: /content/drive/MyDrive/FinIQ_Model

✅ SUCCESS! Model saved to Google Drive!
📁 Total files saved: 7
⭐ Your model is PERMANENTLY saved!
⭐ Location: /content/drive/MyDrive/FinIQ_Model

📊 FINAL RESULTS:


eval_loss: 0.4011
eval_accuracy: 0.8625
eval_f1: 0.8618
eval_precision: 0.8618
eval_recall: 0.8625
eval_runtime: 1.4941
eval_samples_per_second: 267.7110
eval_steps_per_second: 8.7010
epoch: 2.0000

🎉 ALL DONE! YOUR MODEL IS READY!
📂 Model saved at: /content/drive/MyDrive/FinIQ_Model
🚀 Next step: Test your model!


In [ ]:
from transformers import pipeline

print("📂 Loading your trained model from Google Drive...")
model_path = '/content/drive/MyDrive/FinIQ_Model'

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=model_path,
    tokenizer=model_path
)

print("✅ Model loaded successfully!\n")

# Test headlines
test_headlines = [
    "Apple reports record quarterly revenue beating expectations",
    "Tesla stock crashes 15% amid production delays",
    "Microsoft announces stable quarterly earnings",
    "Amazon plans massive expansion with 50,000 new jobs",
    "Bank faces investigation over fraudulent practices",
    "Company maintains dividend despite volatility"
]

print("🧪 TESTING YOUR AI MODEL:\n")
print("=" * 80)

for i, headline in enumerate(test_headlines, 1):
    result = sentiment_analyzer(headline)[0]

    # Updated label_map to match the string labels returned by the pipeline
    label_map = {
        'negative': ('NEGATIVE 📉', 'SELL 🔴'),
        'neutral': ('NEUTRAL ➖', 'HOLD 🟡'),
        'positive': ('POSITIVE 📈', 'BUY 🟢')
    }

    sentiment, signal = label_map[result['label']]
    confidence = result['score'] * 100

    print(f"\n{i}. 📰 {headline}")
    print(f"   Sentiment: {sentiment}")
    print(f"   Signal: {signal}")
    print(f"   Confidence: {confidence:.1f}%")
    print("   " + "-" * 70)

print("\n" + "=" * 80)
print("🎉 Your AI-powered sentiment analyzer is working!")
print("=" * 80)

Device set to use cuda:0


📂 Loading your trained model from Google Drive...
✅ Model loaded successfully!

🧪 TESTING YOUR AI MODEL:


1. 📰 Apple reports record quarterly revenue beating expectations
   Sentiment: NEUTRAL ➖
   Signal: HOLD 🟡
   Confidence: 92.1%
   ----------------------------------------------------------------------

2. 📰 Tesla stock crashes 15% amid production delays
   Sentiment: POSITIVE 📈
   Signal: BUY 🟢
   Confidence: 66.6%
   ----------------------------------------------------------------------

3. 📰 Microsoft announces stable quarterly earnings
   Sentiment: NEUTRAL ➖
   Signal: HOLD 🟡
   Confidence: 48.1%
   ----------------------------------------------------------------------

4. 📰 Amazon plans massive expansion with 50,000 new jobs
   Sentiment: NEUTRAL ➖
   Signal: HOLD 🟡
   Confidence: 73.6%
   ----------------------------------------------------------------------

5. 📰 Bank faces investigation over fraudulent practices
   Sentiment: NEGATIVE 📉
   Signal: SELL 🔴
   Confidence: 77